In [33]:
import pandas as pd
import numpy as np
import spacy as spacy
from tqdm import tqdm_notebook as tqdm

In [2]:
import keras
import sklearn

from sklearn.model_selection import train_test_split

In [3]:
#spacy docs https://spacy.io/usage/v3#migrating-add-pipe

In [5]:
df_train = pd.read_csv('/Users/Kamil/Desktop/koźmiński/Python Notebooks/2nd sem/Final Project/archive-2/Constraint_Train.csv', index_col='id')
df_test = pd.read_csv('/Users/Kamil/Desktop/koźmiński/Python Notebooks/2nd sem/Final Project/archive-2/english_test_with_labels.csv', index_col='id')
df_val = pd.read_csv('/Users/Kamil/Desktop/koźmiński/Python Notebooks/2nd sem/Final Project/archive-2/Constraint_Val.csv', index_col='id')

In [20]:
df_sentiment_train = pd.read_csv("/Users/Kamil/Desktop/koźmiński/Python Notebooks/2nd sem/Final Project/archive/Corona_NLP_test.csv", encoding = "ISO-8859-1")
df_sentiment_test = pd.read_csv("/Users/Kamil/Desktop/koźmiński/Python Notebooks/2nd sem/Final Project/archive/Corona_NLP_train.csv", encoding = "ISO-8859-1")

In [23]:
df_sentiment_test.head(5)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [6]:
df_train

,tweet,label
id,,
1,The CDC currently reports 99031 deaths. In gen...,real
2,States reported 1121 deaths a small rise from ...,real
3,Politically Correct Woman (Almost) Uses Pandem...,fake
4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
5,Populous states can generate large case counts...,real
...,...,...
6416,A tiger tested positive for COVID-19 please st...,fake
6417,???Autopsies prove that COVID-19 is??� a blood...,fake
6418,_A post claims a COVID-19 vaccine has already ...,fake


In [7]:
df_test

,tweet,label
id,,
1,Our daily update is published. States reported...,real
2,Alfalfa is the only cure for COVID-19.,fake
3,President Trump Asked What He Would Do If He W...,fake
4,States reported 630 deaths. We are still seein...,real
5,This is the sixth time a global health emergen...,real
...,...,...
2136,#CoronaVirusUpdates: State-wise details of Tot...,real
2137,Tonight 12(midnight) onwards Disaster Manageme...,fake
2138,296 new cases of #COVID19Nigeria; Plateau-85 E...,real


In [8]:
df_val

,tweet,label
id,,
1,Chinese converting to Islam after realising th...,fake
2,11 out of 13 people (from the Diamond Princess...,fake
3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
4,Mike Pence in RNC speech praises Donald Trump’...,fake
5,6/10 Sky's @EdConwaySky explains the latest #C...,real
...,...,...
2136,Donald Trump wrongly claimed that New Zealand ...,fake
2137,Current understanding is #COVID19 spreads most...,real
2138,Nothing screams “I am sat around doing fuck al...,fake


In [9]:
df = df_train.append(df_test)

In [10]:
df = df.append(df_val)

In [11]:
df

,tweet,label
id,,
1,The CDC currently reports 99031 deaths. In gen...,real
2,States reported 1121 deaths a small rise from ...,real
3,Politically Correct Woman (Almost) Uses Pandem...,fake
4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
5,Populous states can generate large case counts...,real
...,...,...
2136,Donald Trump wrongly claimed that New Zealand ...,fake
2137,Current understanding is #COVID19 spreads most...,real
2138,Nothing screams “I am sat around doing fuck al...,fake


In [12]:
df.iloc[0,0]

'The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.'

In [24]:
maxlen=140

In [25]:
tweets = df["tweet"].to_numpy()

label = df['label'].apply(lambda x: 1 if x == 'real' else 0).to_numpy()

In [26]:
tweets

array(['The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.',
       'States reported 1121 deaths a small rise from last Tuesday. Southern states reported 640 of those deaths. https://t.co/YASGRTT4ux',
       'Politically Correct Woman (Almost) Uses Pandemic as Excuse Not to Reuse Plastic Bag https://t.co/thF8GuNFPe #coronavirus #nashville',
       ...,
       'Nothing screams “I am sat around doing fuck all during lockdown” quite like confident assumption that other people are sat around doing fuck all during lockdown https://t.co/Hs3rZAAl1h',
       'Birx says COVID-19 outbreak not under control because ‘people are on the move’ https://t.co/YFqi6f7Rvj',
       "Another 4422 new coronavirus cases have been confirmed in the UK - the highest daily number since 8 May. It's up from 4322 new cases reported on Friday and the overall total nationwide now 

In [38]:
label

array([1, 1, 0, ..., 0, 0, 1])

In [36]:
def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in tqdm(enumerate(sequences)):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

# Our vectorized training data
df_train_vectorized = vectorize_sequences(df_train)
# Our vectorized test data
df_test_vectorized = vectorize_sequences(df_test)

<ipython-input-36-16e45c851797>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, sequence in tqdm(enumerate(sequences)):


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [37]:
from keras.layers import Embedding
from keras import preprocessing

In [28]:
x_test

NameError: name 'x_test' is not defined

In [43]:
from keras.preprocessing.text import Tokenizer


# We create a tokenizer, configured to only take
# into account the top-1000 most common words
tokenizer = Tokenizer(num_words=25000)
# This builds the word index
tokenizer.fit_on_texts(tweets)

one_hot_results = tokenizer.texts_to_matrix(tweets, mode='binary')

# This turns strings into lists of integer indices.
sequences_tweets = tokenizer.texts_to_sequences(tweets)
word_index = tokenizer.word_index
print('Found {} unique tokens.'.format(len(word_index)))

Found 27417 unique tokens.


In [44]:
word_index

{'the': 1,
 'of': 2,
 't': 3,
 'co': 4,
 'https': 5,
 'to': 6,
 'in': 7,
 'a': 8,
 'and': 9,
 'covid': 10,
 'is': 11,
 '19': 12,
 'for': 13,
 'cases': 14,
 'coronavirus': 15,
 'are': 16,
 'covid19': 17,
 'that': 18,
 'on': 19,
 'from': 20,
 'new': 21,
 'have': 22,
 'with': 23,
 'has': 24,
 'as': 25,
 'we': 26,
 'at': 27,
 'people': 28,
 'this': 29,
 'more': 30,
 'tests': 31,
 'be': 32,
 'by': 33,
 'our': 34,
 'deaths': 35,
 'india': 36,
 'not': 37,
 'been': 38,
 '1': 39,
 'it': 40,
 'there': 41,
 'states': 42,
 '2': 43,
 'number': 44,
 'who': 45,
 'you': 46,
 'total': 47,
 'amp': 48,
 'will': 49,
 'can': 50,
 'confirmed': 51,
 'reported': 52,
 'testing': 53,
 'than': 54,
 'all': 55,
 'was': 56,
 'now': 57,
 'health': 58,
 'or': 59,
 '3': 60,
 'today': 61,
 'an': 62,
 'state': 63,
 'day': 64,
 'they': 65,
 'your': 66,
 'but': 67,
 'if': 68,
 'one': 69,
 'no': 70,
 'indiafightscorona': 71,
 'about': 72,
 'virus': 73,
 'up': 74,
 'pandemic': 75,
 'which': 76,
 'report': 77,
 'vaccine': 78

In [39]:
tweets[0]

'The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.'

In [40]:
sequences_tweets[0]

[1,
 151,
 296,
 279,
 10034,
 35,
 7,
 489,
 1,
 5545,
 7,
 161,
 1507,
 283,
 510,
 2080,
 16,
 811,
 9,
 10035,
 1,
 161,
 756,
 711,
 27,
 2081,
 987,
 28,
 61]

In [80]:
x_train, x_test, y_train, y_test = train_test_split(sequences_tweets, label, test_size=0.1)

In [81]:
x_train[0]

[26,
 411,
 1215,
 8,
 1839,
 2,
 5076,
 20,
 1,
 308,
 800,
 2,
 20659,
 169,
 16,
 2130,
 3709,
 23,
 15,
 82,
 136,
 9,
 1,
 20660,
 2019,
 19,
 1,
 7947,
 5,
 3,
 4,
 20661]

In [96]:
lens =[]
a = 0
for t in sequences_tweets:
    element = []
    element.append(len(t))
    element.append(a)
    element.append(tweets[a])
    a = a + 1
    
    lens.append(element)


In [97]:
lens

[[29,
  0,
  'The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.'],
 [21,
  1,
  'States reported 1121 deaths a small rise from last Tuesday. Southern states reported 640 of those deaths. https://t.co/YASGRTT4ux'],
 [19,
  2,
  'Politically Correct Woman (Almost) Uses Pandemic as Excuse Not to Reuse Plastic Bag https://t.co/thF8GuNFPe #coronavirus #nashville'],
 [29,
  3,
  '#IndiaFightsCorona: We have 1524 #COVID testing laboratories in India and as on 25th August 2020 36827520 tests have been done : @ProfBhargava DG @ICMRDELHI #StaySafe #IndiaWillWin https://t.co/Yh3ZxknnhZ'],
 [45,
  4,
  'Populous states can generate large case counts but if you look at the new cases per million today 9 smaller states are showing more cases per million than California or Texas: AL AR ID KS KY LA MS NV and SC. https://t.co/1pYW6cWRaS'],
 [48,
  5,
  'Covid Act N

In [98]:
df_lens = pd.DataFrame(lens)

In [103]:
df_lens.sort_values(by=0, ascending=False)

,0,1,2
7889,1667,7889,I’ve never had a chance to work on something t...
270,1492,270,"Amoxi"" Capsule made in US is the only medicine..."
1109,873,1109,"Actors"" applauded Macron when he visited the s..."
6926,355,6926,NYC man drops dead in middle of street — suspe...
6306,343,6306,Man visited Albany N.Y. days before dying from...
...,...,...,...
6129,4,6129,WHO changed its directives
1019,3,1019,Cocaine kills coronavirus.
4579,3,4579,Coffee cures coronavirus.
2691,3,2691,Biomagnetism cures coronavirus.


In [106]:
df_lens.iloc[1140,2]

'Pier Luigi Bersani'

Tokenizer do niczego chyba nie służy, bo u leona wsm nic z nim nie robilismy wiec na co on nam?

w notebooku 1.2 robilismy vectorizing, asarray i to tyle, bez tokenizera

próbowaliśmy vectorizing zrobić ale nie działał nam